In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import seaborn as sns
import pandas as pd
from tensorflow.keras.callbacks import TensorBoard
from token import NAME
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import os
import cv2
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [2]:
data_path = pathlib.Path.cwd().joinpath("vehicle2")
categories = ["car", "truck"]

for category in categories:
    path = os.path.join(data_path, category)
    for image in os.listdir(path):
        image_array = cv2.imread(os.path.join(path, image), cv2.IMREAD_GRAYSCALE)

image_height = 128
image_width = 128

new_array = cv2.resize(image_array, (image_width, image_height))



In [3]:
training_data = []

def create_training_data():
    for category in categories:
        path = os.path.join(data_path, category)
        class_num = categories.index(category)
        for image in os.listdir(path):
            try:
                image_array = cv2.imread(os.path.join(path, image), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(image_array, (image_width, image_height))
                training_data.append([new_array, class_num])
            except Exception as e:
                print(os.path.join(path, image))

create_training_data()

In [4]:
import random

random.shuffle(training_data)

In [5]:
x = []
y = []

In [6]:
for features, label in training_data:
    x.append(features)
    y.append(label)

x = np.array(x).reshape(-1, image_height, image_width, 1)
y = np.array(y)
x = x/255.0

In [7]:

tensorboard = TensorBoard(log_dir='carlogs/{}'.format(NAME))

In [8]:
model = Sequential()

model.add(Conv2D(32, (3,3), input_shape = x.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=['accuracy'])

model.fit(x, y, batch_size=32, epochs=10, validation_split=0.2, callbacks=[tensorboard])




Epoch 1/10


255/255 [==============================] - 35s 132ms/step - loss: 0.6426 - accuracy: 0.6287 - val_loss: 0.5690 - val_accuracy: 0.7232
Epoch 2/10
255/255 [==============================] - 33s 129ms/step - loss: 0.5363 - accuracy: 0.7305 - val_loss: 0.7565 - val_accuracy: 0.5973
Epoch 3/10
255/255 [==============================] - 32s 127ms/step - loss: 0.4853 - accuracy: 0.7737 - val_loss: 0.4330 - val_accuracy: 0.8058
Epoch 4/10
255/255 [==============================] - 32s 127ms/step - loss: 0.4036 - accuracy: 0.8226 - val_loss: 0.3819 - val_accuracy: 0.8382
Epoch 5/10
255/255 [==============================] - 32s 127ms/step - loss: 0.3604 - accuracy: 0.8392 - val_loss: 0.3564 - val_accuracy: 0.8559
Epoch 6/10
255/255 [==============================] - 32s 127ms/step - loss: 0.3128 - accuracy: 0.8740 - val_loss: 0.3156 - val_accuracy: 0.8668
Epoch 7/10
255/255 [==============================] - 32s 127ms/step - loss: 0.2821 - accuracy: 0.8827 - val_loss: 0.3159 - v

In [9]:
y_preds = model.predict(x)
y_probs = y_preds.round().astype(int)
y_flat = y_probs.reshape(-1)
cm = confusion_matrix(y, y_flat)

318/318 [==============================] - 11s 34ms/step


In [10]:
def show_car_matrix():
    cmd_obj = ConfusionMatrixDisplay(cm, display_labels=categories)
    cmd_obj.plot()
    cmd_obj.ax_.set(title='Car/Truck Confusion Matrix',
                    xlabel='Predicted',
                    ylabel='Actual')
    plt.show()

In [11]:
model.save('carstrucks.model')

INFO:tensorflow:Assets written to: carstrucks.model\assets


INFO:tensorflow:Assets written to: carstrucks.model\assets


In [ ]:
model = tf.keras.models.load_model('carstrucks.model')

In [ ]:
import requests

In [ ]:
from PIL import Image

In [ ]:
import urllib

In [ ]:
def predictionurl(url):
    req=urllib.request.urlopen(url)
    arr=np.asarray(bytearray(req.read()), dtype=np.uint8)
    img=cv2.imdecode(arr, -1)
    grayimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    image_height = 128
    image_width = 128
    new_array = cv2.resize(grayimg, (image_width, image_height))
    new_array = new_array.reshape(-1, image_height, image_width, 1)
    print(categories[int(model.predict([new_array]))])
    return



In [ ]:
def predictionfile(filename):
    image_height = 128
    image_width = 128
    image_array = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(image_array, (image_width, image_height))
    new_array = new_array.reshape(-1, image_height, image_width, 1)
    print(categories[int(model.predict([new_array]))])
    return